# **Descenso de gradiente**

El Descenso de Gradiente es un algoritmo de optimización muy genérico. La idea general es ajustar los parámetros de forma iterativa para minimizar una función de costo. 

Supón que estás perdido en las montañas; solo puedes sentir la inclinación del suelo bajo tus pies. Una buena estrategia para llegar al fondo del valle rápidamente es ir cuesta abajo en la dirección de la pendiente más pronunciada. Esto es exactamente lo que hace el Descenso de Gradiente: mide el gradiente local de la función de error con respecto al vector de parámetros $\theta$, y avanza en la dirección del gradiente descendente. 

Concretamente, comienzas llenando $\theta$ con valores aleatorios, y luego lo mejoras gradualmente, dando un pequeño paso a la vez (*learning rate*), intentando en cada paso disminuir la función de costo (por ejemplo, el MSE), hasta que el algoritmo converge en un mínimo.

<p align="center">
  <img src="img/gradientDescent01.png" width="400">
</p>

Un parámetro importante en el Descenso de Gradiente es el tamaño de los pasos, determinado por el hiperparámetro llamado tasa de aprendizaje (learning rate). Si la tasa de aprendizaje es demasiado pequeña, el algoritmo tendrá que pasar por muchas iteraciones para converger, pero si la tasa de aprendizaje es demasiado alta, podrías saltar de un lado a otro del valle y terminar en el lado opuesto, posiblemente incluso más arriba de donde estabas antes

<div style="display: flex; justify-content: center; align-items: center; gap: 40px;">
  <img src="img/gradientDescent02.png" alt="Izquierda" width="400">
  <img src="img/gradientDescent03.png" alt="Derecha" width="400">
</div>

No todas las funciones de costo son parabolas. Puede haber agujeros, crestas, mesetas y todo tipo de terrenos irregulares, lo que hace que la convergencia al mínimo sea muy difícil. En la Figura muestra los dos desafíos principales del Descenso de Gradiente: si la inicialización aleatoria comienza el algoritmo a la izquierda, entonces convergerá a un mínimo local, que no es tan bueno como el mínimo global. Si comienza a la derecha, entonces tardará mucho tiempo en atravesar la meseta, y si te detienes demasiado pronto, nunca alcanzarás el mínimo global.

<p align="center">
  <img src="img/gradientDescent04.png" width="400">
</p>

Afortunadamente, la función de costo MSE para un modelo de Regresión Lineal resulta ser una función parabola (convexa), esto implica que no hay mínimos locales, solo un mínimo global.

---

- Cuando utilices el Descenso de Gradiente, debes asegurarte de que todas las características tengan una escala similar (por ejemplo, usando la clase `StandardScaler` de **Scikit-Learn**); de lo contrario, el algoritmo tardará mucho más tiempo en converger.

    ```python
    from sklearn.preprocessing import StandardScaler

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    ```

---

Para implementar el Descenso de Gradiente, necesitas calcular el gradiente de la función de costo con respecto a cada parámetro del modelo $\theta_j$. La siguiente ecuacion calcula la derivada parcial de la función de costo con respecto al parámetro $\theta_j$, denotada como $\frac{\partial}{\partial \theta_j} MSE(\theta)$.

$$
\frac{\partial}{\partial \theta_j} \text{MSE}(\theta) =
\frac{2}{m} \sum_{i=1}^{m} 
\left( \theta^T x^{(i)} - y^{(i)} \right) x_j^{(i)}
$$

En lugar de calcular estas derivadas parciales individualmente, puedes calcularlas todas de una sola vez. El vector gradiente, denotado como $\nabla_\theta MSE(\theta)$, contiene todas las derivadas parciales de la función de costo (una para cada parámetro del modelo).

$$
\nabla_{\theta} \text{MSE}(\theta) =
\begin{pmatrix}
\frac{\partial}{\partial \theta_0} \text{MSE}(\theta) \\
\frac{\partial}{\partial \theta_1} \text{MSE}(\theta) \\
\vdots \\
\frac{\partial}{\partial \theta_n} \text{MSE}(\theta)
\end{pmatrix}
= 
\frac{2}{m} \mathbf{X}^T (\mathbf{X}\theta - \mathbf{y})
$$

Una vez que tienes el vector gradiente, que apunta hacia arriba, simplemente ve en la dirección opuesta para ir hacia abajo. Esto significa restar $\nabla_\theta MSE(\theta)$ de $\theta$. Aquí es donde entra en juego la tasa de aprendizaje $\eta$ (eta): multiplica el vector gradiente por $\eta$ para determinar el tamaño del paso cuesta abajo. La ecuacion de paso del algoritmo descendente de gradiente es:

$$
\theta^{(\text{paso siguiente})} = \theta - \eta \nabla_\theta MSE(\theta)
$$

### Implementacion

In [27]:
import numpy as np
import matplotlib.pyplot as plt

def add_bias(X):
    """Devuelve una nueva matriz con una columna de 1s al principio (X_0=1)."""
    X_b = np.c_[np.ones((X.shape[0], 1)), X]
    return X_b

def gradientDescent(lr,n_epochs,X, y):
    m = X.shape[0]
    n = X.shape[1]
    theta = np.random.randn(n,1)

    for i in range(n_epochs):
        gradients = 2/m * X.T @ (X @ theta - y)
        theta -= lr * gradients

    return theta
    
    

In [28]:
m = 100         # Cantidad de datos
n = 3           # Features

X = 2 * np.random.randn(m, n)
X_b = add_bias(X)

true_theta = np.array([[4], [3], [2], [1]]) 

y = X_b @ true_theta + np.random.rand(m, 1)

theta = gradientDescent(0.1,10,X_b,y)

print("Parámetros aprendidos (θ):")
print(theta)

Parámetros aprendidos (θ):
[[3.92874385]
 [2.96360959]
 [2.00046389]
 [0.99695659]]



## Normalizacion

Para acelerar la convergencia del algoritmo, las características se deben normalizar de modo que $\bar{X_j}=0$ y $\sigma(X_j)=1$, es decir, que los valores de cada variable $X_j$ pasen a tener media 0 y desviación 1. Para ello, se aplica la siguiente expresión.

$$
x_j^{(i)} = \frac{x_j^{(i)}-\bar{X_j}}{\sigma(X_j)}
$$

El algoritmo de la gradiente descendiente devuelve los coeficientes $\alpha$ que minimizan el coste para las variables normalizadas. Por tanto, hay transformarlos para su aplicación sobre las variables en los rangos originales.

# Descenso de gradiente estocástico (SGD)

El principal problema del Descenso de Gradiente por Lotes es el hecho de que utiliza todo el conjunto de entrenamiento para calcular los gradientes en cada paso, lo que lo hace muy lento cuando el conjunto de entrenamiento es grande. En el extremo opuesto, el Descenso de Gradiente Estocástico simplemente elige una instancia aleatoria del conjunto de entrenamiento en cada paso y calcula los gradientes basándose únicamente en esa única instancia.

Funcionamiento:

<p align="center">
  <img src="img/SGD01.png" width="600">
</p>

- En el descenso de gradientes tradicional, los gradientes se calculan en función de todo el conjunto de datos, lo que puede resultar computacionalmente costoso para conjuntos de datos grandes.
- En el descenso de gradiente estocástico, el gradiente se calcula para cada ejemplo de entrenamiento (o un pequeño subconjunto de ejemplos de entrenamiento) en lugar de para todo el conjunto de datos.

La regla de actualización del descenso del gradiente estocástico es:

$$
\nabla_{\theta} \text{MSE}(\theta)_{(i)} = 2 x_i^{T}
\left( x_i \theta - y_{i} \right)
$$

La diferencia clave con el descenso de gradiente tradicional radica en que, en SGD, las actualizaciones de parámetros se basan en un único punto de datos, no en todo el conjunto de datos. La selección aleatoria de puntos de datos introduce estocasticidad.

### Implementacion

In [ ]:
def stochasticGradientDescent(lr0,lr1,n_epochs,X, y):
    m = X.shape[0]
    n = X.shape[1]
    theta = np.random.randn(n,1)

    def learning_schedule(t):
        return lr0 / (t + lr1)
    
    for epoch in range(n_epochs):
        for i in range(m):
            random_index = np.random.randint(m)
            xi = X_b[random_index:random_index+1]
            yi = y[random_index:random_index+1]
            gradients = 2 * xi.T @ (xi @ theta - yi)
            eta = learning_schedule(epoch * m + i)
            theta = theta - eta * gradients
    
    return theta

In [32]:
m = 100         # Cantidad de datos
n = 3           # Features

X = 2 * np.random.randn(m, n)
X_b = add_bias(X)

true_theta = np.array([[4], [3], [2], [1]]) 

y = X_b @ true_theta + np.random.rand(m, 1)

theta = stochasticGradientDescent(5,50,10,X_b,y)

print("Parámetros aprendidos (θ):")
print(theta)

Parámetros aprendidos (θ):
[[4.42724463]
 [3.01872965]
 [1.99368464]
 [1.02676199]]


### Implementacion con sklearn

In [39]:
from sklearn.linear_model import SGDRegressor
from sklearn.preprocessing import StandardScaler

m = 100         # Cantidad de datos
n = 3           # Features

# Datos de ejemplo
X = 2 * np.random.rand(m, n)
y = 4 + 3*X[:,0] + 2*X[:,1] + X[:,2] + np.random.randn(m)

# Escalamos los datos (SGD lo necesita)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Creamos el modelo
sgd_reg = SGDRegressor(max_iter=1000, tol=1e-3, penalty=None, eta0=0.1)

# Entrenamos
sgd_reg.fit(X, y)

print("Intercepto (bias):", sgd_reg.intercept_)
print("Coeficientes (θ):", sgd_reg.coef_)

Intercepto (bias): [3.45517509]
Coeficientes (θ): [3.0345116  1.86385113 1.32201842]


# Mini-batch Gradient Descent

El Mini-batch GD calcula los gradientes en pequeños conjuntos aleatorios de instancias llamados mini-lotes. El progreso del algoritmo en el espacio de parámetros es menos errático que con el SGD, especialmente con mini-lotes relativamente grandes. Como resultado, el Mini-batch GD terminará caminando un poco más cerca del mínimo que el SGD. Pero, por otro lado, puede ser más difícil para este escapar de los mínimos locales.

Para un mini batch de tamaño b:

$$
\nabla_{\theta} \text{MSE}(\theta) =
\begin{pmatrix}
\frac{\partial}{\partial \theta_0} \text{MSE}(\theta) \\
\frac{\partial}{\partial \theta_1} \text{MSE}(\theta) \\
\vdots \\
\frac{\partial}{\partial \theta_b} \text{MSE}(\theta)
\end{pmatrix}
= 
\frac{2}{b} \mathbf{X}^T_B (\mathbf{X}_B\theta - \mathbf{y}_B)
$$

y la actualizacion es:

$$
\theta^{(\text{paso siguiente})} = \theta - \eta \nabla_\theta MSE(\theta)
$$

### Implementacion

In [41]:
def miniBatchGradientDescent(lr,n_epochs,batch_size,X, y):
    m = X.shape[0]
    n = X.shape[1]
    theta = np.random.randn(n,1)
    
    for epoch in range(n_epochs):
        shuffled_indices = np.random.permutation(m)
        X_b_shuffled = X_b[shuffled_indices]
        y_shuffled = y[shuffled_indices]
        for i in range(m):
            X_batch = X_b_shuffled[i:i+batch_size]
            y_batch = y_shuffled[i:i+batch_size]
            gradients = 2/batch_size * X_batch.T @ (X_batch @ theta - y_batch)
            theta = theta - lr * gradients
    
    return theta

In [42]:
m = 100         # Cantidad de datos
n = 3           # Features

X = 2 * np.random.randn(m, n)
X_b = add_bias(X)

true_theta = np.array([[4], [3], [2], [1]]) 

y = X_b @ true_theta + np.random.rand(m, 1)

theta = miniBatchGradientDescent(0.1,20,10,X_b,y)

print("Parámetros aprendidos (θ):")
print(theta)

Parámetros aprendidos (θ):
[[4.61781576]
 [2.90903395]
 [2.09292983]
 [1.05783127]]
